# CNN Model with Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#user defined module
! pip install alibi
!cat '/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/load_data.py'
import sys
import os
sys.path.append('/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/')
import load_data
from importlib import reload
#standard packages
import numpy as np
import pandas as pd
#Sk-learn functions
from sklearn.model_selection import train_test_split
#Utility
from collections import Counter
#NLTK
import nltk
#tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#integrated gradients
from alibi.explainers import IntegratedGradients

np.random.seed(77)

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

import pandas as pd
import numpy as np
import glob
import os
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ET




def extract_filename(file):
    base = os.path.basename(file)
    filename = os.path.splitext(base)[0]
    return filename

def get_tweets(xml_file):
    '''make a list of all tweets in an xml file.'''
    mytree = ET.parse(xml_file)
    myroot = mytree.getroot()
    tweets = []
    for x in myroot[0].findall('document'):
        tweet = x.text
        tweets.append(tweet)
    return tweets

def read_all_data(path_to_data):
    """Reads all data from XML and Text into Python"""
    path_to_text = path_to_data + 'truth.txt'
    labels = pd.read_csv(path_to_text, delimiter = "\:::",header = None)
    labels.rename(columns={0: 'id', 1: 'bot', 2: 'sex'}, inplace=True)
    path_to_xml = path_to_data + '/*.xml'
    xml = glob.glob(path_to_xml, recursive=False)
    ids = []
    for i in range(len(xml)):
        file = xml[i]
        file_id = extract_filename(

### Load all data

In [ ]:
reload(load_data)
path_to_train = '/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/pan_bot_id/pan19-author-profiling-training-2019-02-18/en/'
train_data = load_data.read_all_data(path_to_train)

path_to_early_bird = '/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/pan_bot_id/pan19-author-profiling-earlybirds-20190320/en/'
early_bird_data = load_data.read_all_data(path_to_early_bird)

path_to_test = '/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/pan_bot_id/pan19-author-profiling-test-2019-04-29/en/'
test_data = load_data.read_all_data(path_to_test)

# X_train_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/X_train.csv')
# X_dev_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/X_dev.csv')
# X_test_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/X_test.csv')
# X_earlybird_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/X_earlybird.csv')
# y_train_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/y_train.csv')
# y_dev_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/y_dev.csv')
# y_test_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/y_test.csv')
# y_early_bird_base = pd.read_csv('/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/all_data/y_earlybird.csv')

/content/gdrive/My Drive/w266_bot_id_final_project/bot_id_pan/load_data.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  labels = pd.read_csv(path_to_text, delimiter = "\:::",header = None)


## Converts Bot Column into a Binary Variable

In [ ]:
def convert_binary_str_text(column):
    if column == "bot":
        return 1
    else:
        return 0

train_data['bot'] = train_data['bot'].apply(convert_binary_str_text)
early_bird_data['bot'] = early_bird_data['bot'].apply(convert_binary_str_text)
test_data['bot'] = test_data['bot'].apply(convert_binary_str_text)


In [ ]:
# y_train_base = y_train_base['bot']
# X_train_base = X_train_base['tweets']
# y_early_bird_base = y_dev_base['bot']
# X_earlybird_base = X_dev_base['tweets']
# y_test_base = y_test_base['bot']
# X_test_base = X_test_base['tweets']


y_train_base = train_data['bot']
X_train_base = train_data['tweets']
y_early_bird_base = early_bird_data['bot']
X_earlybird_base = early_bird_data['tweets']
y_test_base = test_data['bot']
X_test_base = test_data['tweets']
X_train_base, X_dev_base, y_train_base, y_dev_base = train_test_split(X_train_base, y_train_base, test_size = 0.33, random_state = 37)

In [ ]:
#convert data to csv
# X_train_base.to_csv('X_train.csv')
# X_dev_base.to_csv('X_dev.csv')
# y_train_base.to_csv('y_train.csv')
# y_dev_base.to_csv('y_dev.csv')
# X_earlybird_base.to_csv('X_earlybird.csv')
# y_early_bird_base.to_csv('y_earlybird.csv')
# y_test_base.to_csv('y_test.csv')
# X_test_base.to_csv('X_test.csv')


### Checking Max Sequence Length

In [ ]:
train_data['Num_words_text'] = train_data['tweets'].apply(lambda x:len(str(x).split())) 
test_data['Num_words_text'] = test_data['tweets'].apply(lambda x:len(str(x).split())) 

max_train_sentence_length  = train_data['Num_words_text'].max()
max_test_sentence_length  = test_data['Num_words_text'].max()

# X_train_base['Num_words_text'] = X_train_base['tweets'].apply(lambda x:len(str(x).split())) 
# X_test_base['Num_words_text'] = X_test_base['tweets'].apply(lambda x:len(str(x).split())) 

# max_train_sentence_length  = X_train_base['Num_words_text'].max()
# max_test_sentence_length  = X_test_base['Num_words_text'].max()

print('Train Max Sentence Length :'+str(max_train_sentence_length))
print('Test Max Sentence Length :'+str(max_test_sentence_length))


Train Max Sentence Length :97
Test Max Sentence Length :90


### Sets number of words for tokenizer and check if working

In [ ]:
NUM_WORDS=100000
tokenizer = Tokenizer(num_words=NUM_WORDS,lower=True, oov_token="unk")
tokenizer.fit_on_texts(X_train_base)

print(str(tokenizer.texts_to_sequences(['xuzz how are you'])))

[[35512, 63, 23, 14]]


In [ ]:
sequences_train = tokenizer.texts_to_sequences(X_train_base)
sequences_dev=tokenizer.texts_to_sequences(X_dev_base)
sequences_test = tokenizer.texts_to_sequences(X_test_base)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 373131 unique tokens.


### Pads Sequences and Turns y variable to categorical

In [ ]:
X_train = pad_sequences(sequences_train, maxlen = 100)
X_dev = pad_sequences(sequences_dev,maxlen=100)
X_test = pad_sequences(sequences_test,maxlen = 100)

Y_train = to_categorical(np.asarray(y_train_base))
Y_dev = to_categorical(np.asarray(y_dev_base))
Y_test = to_categorical(np.asarray(y_test_base))

print('Shape of X train and X dev tensor and X test tensor:', X_train.shape,X_dev.shape, X_test.shape)
print('Shape of label train and dev tensor and test tensor:', Y_train.shape,Y_dev.shape, Y_dev.shape)

Shape of X train and X dev tensor and X test tensor: (276040, 100) (135960, 100) (264000, 100)
Shape of label train and dev tensor and test tensor: (276040, 2) (135960, 2) (135960, 2)


## Load pre-trained word vectors from google news

In [ ]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path = True)
word_vectors = KeyedVectors.load_word2vec_format(path, binary=True)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
word_vectors['great']

array([ 7.17773438e-02,  2.08007812e-01, -2.84423828e-02,  1.78710938e-01,
        1.32812500e-01, -9.96093750e-02,  9.61914062e-02, -1.16699219e-01,
       -8.54492188e-03,  1.48437500e-01, -3.34472656e-02, -1.85546875e-01,
        4.10156250e-02, -8.98437500e-02,  2.17285156e-02,  6.93359375e-02,
        1.80664062e-01,  2.22656250e-01, -1.00585938e-01, -6.93359375e-02,
        1.04427338e-04,  1.60156250e-01,  4.07714844e-02,  7.37304688e-02,
        1.53320312e-01,  6.78710938e-02, -1.03027344e-01,  4.17480469e-02,
        4.27246094e-02, -1.10351562e-01, -6.68945312e-02,  4.19921875e-02,
        2.50000000e-01,  2.12890625e-01,  1.59179688e-01,  1.44653320e-02,
       -4.88281250e-02,  1.39770508e-02,  3.55529785e-03,  2.09960938e-01,
        1.52343750e-01, -7.32421875e-02,  2.16796875e-01, -5.76171875e-02,
       -2.84423828e-02, -3.60107422e-03,  1.52343750e-01, -2.63671875e-02,
        2.13623047e-02, -1.51367188e-01,  1.04003906e-01,  3.18359375e-01,
       -1.85546875e-01,  

In [ ]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

from tensorflow.keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [ ]:
del(word_vectors)

### Develop Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten, Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [ ]:
sequence_length = 100
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5


inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout) #should this be sigmoid?


In [ ]:
# this creates a model that includes
model = Model(inputs, output)

In [ ]:
#change batch size back to 16 for actual model, done to speed it up
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, Y_train, batch_size=26, epochs=3, verbose=1, validation_data=(X_dev, Y_dev), 
          callbacks = callbacks)

Epoch 1/3
10617/10617 [==============================] - 1737s 162ms/step - loss: 0.4764 - acc: 0.8653 - val_loss: 0.3102 - val_acc: 0.9242
Epoch 2/3
10617/10617 [==============================] - 1706s 161ms/step - loss: 0.2784 - acc: 0.9337 - val_loss: 0.2688 - val_acc: 0.9319
Epoch 3/3
10617/10617 [==============================] - 1729s 163ms/step - loss: 0.2338 - acc: 0.9442 - val_loss: 0.2534 - val_acc: 0.9319


In [ ]:
y_hat = model.predict(X_test)

In [ ]:
# score, acc = model.evaluate(X_test, Y_test,
#                             batch_size=100)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 100, 300, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 98, 1, 100)   90100       reshape[0][0]                    
______________________________________________________________________________________________

In [ ]:
model_fname = 'CNN'
my_wd = '/content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/'

model.save(os.path.join(my_wd, model_fname))

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/w266_bot_id_final_project/bot_id_pan/CNN/assets


In [ ]:
model.save("cnn.h5")

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('cnn.h5')

### Accuracy for CNN was 78%